This notebook starts to work through the different codelists needed by the upgraded LIMS and its new sample management module replacing an older custom application (WBSSS). It looks at the different data tables used to store pick list values from the legacy system and explores how we can organize some or all of these as concepts within the GeoKB. It does require specialized access to some internal tables and is intended for operation by specific USGS personnel involved in the work.

In [47]:
import pandas as pd
import requests
import sys
import pyodbc
import rdflib

sys.path.append('../harvesters')

from wbmaker import WikibaseConnection

geokb = WikibaseConnection('GEOKB_CLOUD')

In [15]:
geokb_rocks_query = """
PREFIX wd: <https://geokb.wikibase.cloud/entity/>
PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>

SELECT ?rock ?rockLabel ?rock_alt_label ?rockDescription ?subClassOf ?subClassOfLabel ?same_as
WHERE {
  ?rock wdt:P2* wd:Q41261 ;
        wdt:P2 ?subClassOf ;
        wdt:P84 ?same_as .
  OPTIONAL {
    ?rock skos:altLabel ?rock_alt_label .
    FILTER (lang(?rock_alt_label)='en')
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
"""

geokb_rocks = geokb.sparql_query(geokb_rocks_query)

In [39]:
import pandas_access as mdb

for tbl in mdb.list_tables("../data/WBSSS_Lookup_Tables.accdb"):
    print(tbl)

SSF_LU_ALTERATION
SSF_LU_ANALYSIS
SSF_LU_ANALYSIS_BACKUP
SSF_LU_ASH
SSF_LU_ASHED
SSF_LU_BLEND
SSF_LU_BLENDED
SSF_LU_CLASSES
SSF_LU_COMPLETED_TREATMENT
SSF_LU_CONCENTRATED
SSF_LU_DATE_FORMAT
SSF_LU_DATUM
SSF_LU_DEPOSIT_ENVIRON
SSF_LU_DRIED
SSF_LU_FACIES_GRADE
SSF_LU_FREEZE_DRIED
SSF_LU_GEOLOGIC_AGE
SSF_LU_GRIND
SSF_LU_GROUND
SSF_LU_LAND_COVER
SSF_LU_LEACH_EXTRACTED_PHASE
SSF_LU_LEACH_EXTRACTION_REAGENT
SSF_LU_LEACH_METHOD_COLLECT
SSF_LU_LEACH_O_MAT_SAMP_SOURCE
SSF_LU_LEACH_O_MAT_SAMP_TYPE
SSF_LU_LEACH_PARTICLE_SIZE
SSF_LU_LEACH_SAMP_PRES_METH
SSF_LU_LEACH_SAMPLE_SOURCE
SSF_LU_LEACH_SAMPLE_TYPE
SSF_LU_LEACH_TEMPERATURE
SSF_LU_LEACH_TIME
SSF_LU_LIMS_PREP_REQUEST
SSF_LU_LOCATIONS
SSF_LU_METAMORPHISM
SSF_LU_METHOD_COLLECTED
SSF_LU_MILL
SSF_LU_MILLED
SSF_LU_MIN_SPECIFIC_NAME_COPY
SSF_LU_MINERAL_SOURCE
SSF_LU_MINERALIZATION
SSF_LU_MINHOST
SSF_LU_MISC_SOLID_SPECIFIC_NAME
SSF_LU_MISC_SOURCE
SSF_LU_MISC_SPECIFICNAME
SSF_LU_MISCMASTER
SSF_LU_NEW_COLL_PROTOCOLS
SSF_LU_ORG_COAL_COMP_TREATMENT
SSF_L

In [46]:
mdb.read_table("../data/WBSSS_Lookup_Tables.accdb", "SSF_LU_GEOLOGIC_AGE")

,GEOLOGIC_AGE,SORT_ORDER,SEQ
0,unknown,30.0,27
1,precambrian undifferentiated,29.0,26
2,proterozoic undifferentiated,26.0,23
3,neoproterozoic,23.0,24
4,mesoproterozoic,24.0,25
5,paleozoic undifferentiated,22.0,22
6,cambrian,21.0,21
7,ordovician,20.0,20
8,silurian,19.0,19
9,devonian,18.0,18


In [8]:
lims_rocks = pd.read_excel('../data/SSF_LU_SPECIFIC_NAME.xlsx')

In [12]:
lims_rocks[~lims_rocks['SPECIFIC_NAME'].isin(geokb_rocks['rockLabel'])]

,SORT_ORDER,PRIMARY_CLASS,SECONDARY_CLASS,SPECIFIC_NAME,DESCRIPTION,SEQ
0,1,rock,igneous,igneous unidentified,rock > igneous,28
3,5,rock,igneous,intrusive,rock > igneous > plutonic > intrusive,31
6,13,rock,igneous,syenodiorite,rock > igneous > plutonic > dioritoid > diorit...,34
12,22,rock,igneous,diabase,rock > igneous > plutonic > gabbroid > diabase,40
13,23,rock,igneous,dolerite,rock > igneous > plutonic > gabbroid > dolerite,41
...,...,...,...,...,...,...
272,711,rock,miscellaneous,nodule,rock > miscellaneous > nodule,21
273,712,rock,miscellaneous,ochre,rock > miscellaneous > ochre,22
275,714,rock,miscellaneous,psilomelane,rock > miscellaneous > psilomelane,24
278,720,rock,sedimentary,paleosol,rock > sedimentary > clastic > paleosol,288


In [34]:
geokb_mapping = geokb_rocks[geokb_rocks['rockLabel'].notnull()].drop_duplicates().set_index('rockLabel')['rock'].to_dict()
geokb_mapping.update(geokb_rocks[geokb_rocks['rock_alt_label'].notnull()].drop_duplicates().set_index('rock_alt_label')['rock'].to_dict())

gso_mapping = geokb_rocks[geokb_rocks['same_as'].str.contains('w3id.org')].drop_duplicates().set_index('rockLabel')['same_as'].to_dict()
gso_mapping.update(geokb_rocks[geokb_rocks['same_as'].str.contains('w3id.org') & geokb_rocks['rock_alt_label'].notnull()].drop_duplicates().set_index('rock_alt_label')['same_as'].to_dict())

mindat_mapping = geokb_rocks[geokb_rocks['same_as'].str.contains('mindat.org')].drop_duplicates().set_index('rockLabel')['same_as'].to_dict()
mindat_mapping.update(geokb_rocks[geokb_rocks['same_as'].str.contains('mindat.org') & geokb_rocks['rock_alt_label'].notnull()].drop_duplicates().set_index('rock_alt_label')['same_as'].to_dict())

In [28]:
lims_rocks['geokb_link'] = lims_rocks['SPECIFIC_NAME'].map(geokb_mapping)
lims_rocks['gso_link'] = lims_rocks['SPECIFIC_NAME'].map(gso_mapping)
lims_rocks['mindat_link'] = lims_rocks['SPECIFIC_NAME'].map(mindat_mapping)

In [31]:
lims_rocks

,SORT_ORDER,PRIMARY_CLASS,SECONDARY_CLASS,SPECIFIC_NAME,DESCRIPTION,SEQ,geokb_link,gso_link,mindat_link
0,1,rock,igneous,igneous unidentified,rock > igneous,28,NaN,NaN,NaN
1,2,rock,igneous,aphanite,rock > igneous > aphanitic > aphanite,29,https://geokb.wikibase.cloud/entity/Q43299,https://w3id.org/gso/rockmaterial/aphanite,https://mindat.org/min-50628.html
2,3,rock,igneous,felsite,rock > igneous > aphanitic > felsite,30,https://geokb.wikibase.cloud/entity/Q41790,NaN,https://mindat.org/min-48445.html
3,5,rock,igneous,intrusive,rock > igneous > plutonic > intrusive,31,NaN,NaN,NaN
4,10,rock,igneous,dioritoid,rock > igneous > plutonic > dioritoid,32,https://geokb.wikibase.cloud/entity/Q41608,https://w3id.org/gso/rockmaterial/dioritoid,https://mindat.org/min-48224.html
...,...,...,...,...,...,...,...,...,...
276,715,rock,miscellaneous,sinter,rock > miscellaneous > sinter,25,https://geokb.wikibase.cloud/entity/Q41428,NaN,https://mindat.org/min-49378.html
277,716,rock,miscellaneous,tektite,rock > miscellaneous > tektite,26,https://geokb.wikibase.cloud/entity/Q41390,NaN,https://mindat.org/min-10859.html
278,720,rock,sedimentary,paleosol,rock > sedimentary > clastic > paleosol,288,NaN,NaN,NaN
279,725,rock,sedimentary,coal,rock > sedimentary > organic > coal,289,https://geokb.wikibase.cloud/entity/Q41370,https://w3id.org/gso/rockmaterial/coal,https://mindat.org/min-9353.html


In [30]:
lims_rocks.fillna('').to_excel('../data/SSF_LU_SPECIFIC_NAME_linkages.xlsx', index=False)